In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from functools import partial
from difflib import SequenceMatcher

from konlpy.tag import Twitter
import nltk
from collections import Counter

import requests
from bs4 import BeautifulSoup, Tag

import gensim
from gensim.models import Word2Vec

from multiprocessing import Pool

In [2]:
def extract_by_word_class(poses, word_class):
    global except_words
    result = []
    for word, pose in poses:
        if pose in word_class and not word in except_words:
            result.append(word)
    return result

In [ ]:
# 한국어 Stopwords를 가져옴
res = requests.get("https://www.ranks.nl/stopwords/korean",verify=False)
bsObj = BeautifulSoup(res.text,'html.parser')
contents=[]
for x in bsObj.tr.find_all('td'):
    contents.extend([content for content in x.contents if not isinstance(content,Tag)])

In [3]:
stopwords = pd.read_csv("../data/stopwords.csv")
except_words = set(stopwords.stopwords.values)

**Twitter.pos** - 트위터 A Part-Of-Speech Tagger

**norm - 정규화(normalization)**
    
    한국어를 처리하는 예시입니닼ㅋㅋ -> 한국어를 처리하는 예시입니다ㅋㅋ

**stem - 어근화(stemming)**

    한국어를 처리하는 예시입니다 ㅋㅋ -> 한국어Noun, 를Josa, 처리Noun, 하다verb, 예시Noun, 이다Adjective, ㅋㅋKorean

In [4]:
merge_df = pd.read_csv('../data/merge.csv', delimiter="▒",engine='python')
twitter = Twitter()
# target_atc = merge_df.loc[(merge_df.review_grade<3)&(merge_df.cat4_title=='일반의자'),"review_atc"]

# cat_sentence = target_atc.str.cat(sep=" ")
# poses = twitter.pos(cat_sentence,norm=True,stem=True)
# words = extract_by_word_class(poses,['Noun'])
# Counter(words).most_common(50);

## Word2Vec

#### Reference

1. https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/
2. http://blog.theeluwin.kr/post/146591096133/%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec
3. https://github.com/lovit/soynlp

### Word2Vec with Twitter Tokenizer

https://shuuki4.wordpress.com/2016/01/27/word2vec-%EA%B4%80%EB%A0%A8-%EC%9D%B4%EB%A1%A0-%EC%A0%95%EB%A6%AC/

In [6]:
?twitter.morphs

In [7]:
import time

In [8]:
start_time = time.time()
review_morph = merge_df.review_atc.apply(partial(twitter.morphs,norm=True,stem=True))
sentences = review_morph.values
print('consumed time ---- {}'.format(time.time()-start_time))

consumed time ---- 261.6823742389679


In [9]:
embedding_model = Word2Vec.load("../model/twitter_word2vec.em")

In [10]:
print("디자인---------\n")
print(embedding_model.wv.most_similar(positive=['디자인'],topn=5))
print("\n가격----------\n")
print(embedding_model.wv.most_similar(positive=['가격'],topn=5))
print("\n배송----------")
print(embedding_model.wv.most_similar(positive=['배송'],topn=5))

디자인---------

[('디쟌', 0.6812553405761719), ('색상', 0.5739611387252808), ('모양', 0.5732818841934204), ('색감', 0.5675528049468994), ('성', 0.5010105967521667)]

가격----------

[('금액', 0.7891191244125366), ('품질', 0.6748444437980652), ('퀄리티', 0.6237465143203735), ('이정', 0.5938099026679993), ('저렴', 0.5501326322555542)]

배송----------
[('배달', 0.7343376874923706), ('시공', 0.5897732973098755), ('해피', 0.5793194770812988), ('약속한', 0.5643859505653381), ('약속', 0.5541044473648071)]


In [ ]:
embedding_model.save("../model/twitter_word2vec.em")

### Word2Vec with soynlp Tokenizer

In [ ]:
from soynlp.word import WordExtractor
from soynlp.tokenizer import RegexTokenizer, LTokenizer, MaxScoreTokenizer

In [ ]:
atcs = []
for atc in df.review_atc:
    atcs.append(atc)

In [ ]:
word_extractor = WordExtractor(min_count=500,
                               min_cohesion_forward=0.5, 
                               min_right_branching_entropy=0.0)

In [ ]:
word_extractor.train(atcs)
words = word_extractor.extract()

In [ ]:
word_score_df = pd.DataFrame()
for word, score in words.items():
    word_score = score.cohesion_forward*np.exp(score.right_branching_entropy)

    row = {"word":word, 
    "word_score":word_score,
    "freq":score.leftside_frequency,
    "cohesion":score.cohesion_forward, 
    "branching_entropy":score.right_branching_entropy}
    
    word_score_df = word_score_df.append(row,ignore_index=True) 

In [ ]:
word_score_df.sort_values('word_score',ascending=False)[:50]

In [ ]:
scores = pd.Series(word_score_df.word_score.values,index=word_score_df.word.values).to_dict()

In [ ]:
tokenizer = LTokenizer(scores=scores)

### 성능은 Twitter가 더 좋다고 판단되어짐

### scikit-Learn의 문서 전처리 기능
    https://datascienceschool.net/view-notebook/3e7aadbf88ed4f0d87a76f9ddc925d69/

## 문장 분류하기

https://ratsgo.github.io/natural%20language%20processing/2017/03/08/word2vec/

In [12]:
def calculate_score_mat(vec_keywords,not_keywords, vec_size, except_words, embedding_model):
    embed_df = pd.DataFrame(
        embedding_model.wv.most_similar(
            positive=vec_keywords,
            negative=not_keywords,
            topn=vec_size))
    embed_df.columns = ['word','score']
    embed_df = embed_df[~embed_df.word.isin(set(except_words))]
    exist_words = set(sentences[0]) & set(embed_df.word)
    word_vec = embed_df.word.isin(exist_words).values.astype(np.int)

    word_list = []
    for index, sentence in enumerate(sentences):
        exist_words = set(sentence) & set(embed_df.word)
        word_vec = embed_df.word.isin(exist_words).values.astype(np.int)
        word_list.append(word_vec)

    sentence_mat = np.vstack(word_list)
    score_vec = embed_df.score.values
    score_mat = np.dot(sentence_mat,score_vec)
    return score_mat

### 키워드 score

In [22]:
vec_size = 100
vec_keywords = {'디자인','모양','질감','재질','무늬','소재','외관','형태'}
neg_keywords = {'색감','색깔','색상','색','칼라'}#{'디자인','모양','질감','재질','무늬','소재','외관'}
embed_df = pd.DataFrame(embedding_model.wv.most_similar(positive=vec_keywords,negative=neg_keywords,topn=vec_size))
embed_df.columns = ['word','score']
print(list(embed_df.word))
#print(set(embed_df.word)-set(STOPWORDS))

['재료', '스펀지', 'mdf', 'MDF', '지지', '목재', '구조', '자재', '나무', '합판', '부위', '쇠', '마감', '특유', '구성', '플라스틱', '고무', '판', '피스', '메쉬', '(?)', '종이', '판자', '스폰지', '철제', '기둥', '본드', '조각', '시트', '저가', '등판', '튼튼함', '테이프', '부분', '기능', '구성은', '텅', '톱밥', '질도', '구도', '얇은', '덧', '너트', '이음', '두툼', '박음질', '보강', '아무래도', '약하고', '제질', '두꺼운', '단추', '상태', '까진', '비닐', '충격', '퀄리티', '꼼꼼하고', '용접', '문제', '만들어져', '뼈대', '화학', '얼마나', '단단해서', '중국산', '코팅', '철', '결', '자라', '떨어진', '굉장히', '프레임', '상당히', '특성', '경우', '비었', '견고', '마무리', '지탱', '망가지', '단단히', '상판', '심하고', '몸', '일부', '필름', '볼트', '천이', '지는', '실밥', '도장', '고정', '뒷면', '위아래', '싸구려', '착석', '바느질', '메이드', '느낀']


## 색깔

In [15]:
start_time = time.time()
color_vec_size = 200
color_vec_keywords = {'색감','색깔','색상','색','칼라','컬러','톤'}
color_neg_keywords = {'모양'}
color_score_mat = calculate_score_mat(color_vec_keywords,color_neg_keywords,color_vec_size,except_words,embedding_model)
merge_df['tag_color_score'] = color_score_mat
print(time.time()-start_time)

32.87043738365173


## 디자인

In [32]:
design_vec_size = 200
design_vec_keywords = {'디자인','모양','질감','재질','무늬','소재','외관','형태'}
design_neg_keywords = {'색감','색깔','색상','색','칼라'}
design_score_mat = calculate_score_mat(design_vec_keywords,design_neg_keywords,design_vec_size,except_words,embedding_model)
merge_df['tag_design_score'] = design_score_mat

## 서비스

In [33]:
service_vec_size = 150
service_vec_keywords = ["배송","배달", "택배", "기사", "예약", "출고", "도착", "상담","콜센터", "친절", "응대",'서비스',"as","AS","조치"]
service_neg_keywords = []
service_score_mat = calculate_score_mat(service_vec_keywords,service_neg_keywords,service_vec_size,except_words,embedding_model)
merge_df['tag_service_score'] = service_score_mat

## 조립

In [34]:
assembly_vec_size = 100
assembly_vec_keywords = ['조립','만들기','설치','난이도','어려움','쉬움']
assembly_neg_keywords = []
assembly_score_mat = calculate_score_mat(assembly_vec_keywords,assembly_neg_keywords,assembly_vec_size,except_words,embedding_model)
merge_df['tag_assembly_score'] = assembly_score_mat

## 가격

In [35]:
price_vec_size = 100
price_vec_keywords = [ '퀄리티', '가격','금액','돈','비용','저렴','값','고가','저가','값어치','싼','비싼','싼값','싸구려']
price_neg_keywords = ['배송','배송비','배달']
price_score_mat = calculate_score_mat(price_vec_keywords,price_neg_keywords,price_vec_size,except_words,embedding_model)
merge_df['tag_price_score'] = price_score_mat

In [48]:
print("design 관련 리뷰 갯수 : {}".format(merge_df.loc[merge_df.tag_design_score > .5].shape[0]))
print(list(merge_df.loc[merge_df.tag_design_score > .5,'review_atc'][:100]))

design 관련 리뷰 갯수 : 17382
['다른 상품평에서 본것과 같이 찌그러짐이 장난아닙니다ㅠ펴서 조립을할지 교환을할지 반품을할지 고민중ㅠㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ다시받아도 안찌그러질거란 보장이 없어서그냥 조립했네요~특별히 많이 찌그러진게 상판 뒷쪽이라 사진처럽 웁니다ㅠ 티비 놓으니 표는 안나는데아래 셋탑박스 놓은부분이 울퉁불퉁합니다ㅡ멀리서 본 전체적인 모습은 이쁘구요ㅡ아래 수납도 많이 될듯하네요', '마지막 윗판 얹는 것 땜에 약간 애먹었지만 조립은 크게 어렵지 않았어요.문제는 문을 열 때마다 고정쇠가 빠져서 문을 확 열 수 없다는 점?이 좀 불편하긴 하지만 깔끔하고 예쁩니다.', '생각보다 커서 놀랐어요 근데 어렵기보단 좀 무거워서 여자 혼자 조립하긴 좀 낑낑거리기도 하고 시간도 좀 걸릴것 같긴 해요 그리고 상단판 가운데부분 찌그러져 와서 난감했어요', '이케아꺼 구매할려다가 그나마 조금 저렴한 여기꺼 구매한거고.. 배송비만 2만원짜린데 진짜 장난치는것도 아니고;; 상판이 다 찌글짜글해서 조립하는데 애먹었습니다. 한쪽 문짝은 너덜거리네요.. 그냥 싼마이에 샀다고하고 그냥 쓰겠습니다만 이딴식으로 장사하지마세요.. 리뷰를 안보고 구매해서 후회중이네요;; 정상적으로 제품온 사람이 단 1명도 없는거 같네요ㅋㅋ 배송만 뒤지게빠르고 중국산대비 비싸고 제품품질은 최악 수준', '마켓비 제품은 완제품 아니면 절대 안삽니다. 구매하시려는 분들 이 글보고 참고하세요. 리뷰안남기려고 했는데 오늘 교환제품받아보고 어이없어서 글 쓰네요. 꼭 참고하세요. 일단 마켓비 제품 세개를 구매 했습니다. 11번가에서 한개 마켓비 공식쇼핑몰에서 한개 ㅈㅁㅋ에서 한개 이렇게 세개를 주문했는데요. 찌그러져서 오는건 기본입니다. 전 공구 다를줄알고 가구도 취미로 만들고 하는 남성이라서 그나마 찌그러진부분 펴가며 안맞는부분 맞춰가며 조립했는데요. 여자분들은 아마 찌그러진 상태와 부분에 따라 조립이 불가능하거나 힘드실겁니다. 이게 조립전에 찌그러진 상태를 확인해야 전체교환이 가

In [47]:
print("color 관련 리뷰 갯수 : {}".format(merge_df.loc[merge_df.tag_color_score > .5].shape[0]))
print(list(merge_df.loc[merge_df.tag_color_score > .5,'review_atc'][:100]))

color 관련 리뷰 갯수 : 30971
['티비 선반으로 두려고 샀는데 색도 옅은 그레이라 어디에둬도 잘 어울리고 크기도 적당해요 플스올려놓으니까 귀여워요 나름', '배송 매우 빨랐구요 여자혼자 조립하기에도 다른상품보다 훨씬 수월하네오 화이트가 깨끗해보이고 세련되 보여요 같이 주문한 다른상품들도 만족입니다', '색상 디자인 사이즈 모두 만족스럽네요 생각보다 튼튼해서 오래 쓸듯해요~^^', '비용도저렴하고인테리어효과도있고 작은집에 딱이네요 ㅎㅎ이뻐요 많이 파세요 ㅎㅎ색상은 밝아요 ㅎㅎ', '조립은 힘들었어도 이렇게 뿌듯할 수가 없네요. 깔끔해서 예뻐요', '디자인은 화면과 같이 예뻐요~무광이라 더 맘에 들고요 조립도 무게감빼고는 쉬워요 단 철제용이고 무광이라서 긁힘이 심하니 조립시 주의해야함니다 그리고 상판이 양쪽 찌그러져서 사진찍어 보내고 교환받아 다시 조립하는 번거로움이 있었네요그리고  배송기사님  배송예정일이라 하루종일 집에서 기다렸더니 저녁 9시 다되서 경비실에 맡겼다고 문자 하나오네요 가벼운 제품같으면 바쁘신가보다 하고 생각할텐데 여자혼자 못들어 담날저녁이나 되서야  신랑퇴근후 받아왔네요 홈쇼핑에서 주문하는건 집앞까지 배송이 기본아닌가요?  특히 무거운 제품은 무조건~~', '크기도 딱 좋고 쫌 크긴하지만 완전 깔끔해서 직은방이 커보이는 효과도 잇는것같구 크기대비 깔끔해요좋네요', '넘~이뻐요~조립도 쉽고 하얀것이 인테리효과도 짱이고~ㅎㅎ 분위기가 확 살아요~벽지가 흰색이라보니 넘 잘어울리는것같아요 ㅎㅎ 감사합니다.수납도 이것저것 잡동사니 많이 들어가요~', '매트화이트 주문했는데 색상 이쁘네요.혼자 조립하는거 안어려워요..ㅎㅎ무거워서 낑낑대기는 했지만^^', '가성비 짱이구요 조립하는것도 의외로 쉬웟어요ㅎㅎ티비 위에 올려놧을시 위치도 딱 알맞고 거실에 두고 쓰는데 인테리어 효과도있고너무너무대만족입니다!!', '색도 아주이쁘고 조립하기도 쉽고, 뚜껑부분 맞출때 쪼금 애먹음. 맘에 쏘오오오옥듭니다요. 굳굳', '배송 빨라용다크 그레이는 마켓비만 ㅇㅆ는줄

In [49]:
print("조립 관련 리뷰 갯수 : {}".format(merge_df.loc[merge_df.tag_assembly_score > .5].shape[0]))
print(list(merge_df.loc[merge_df.tag_assembly_score > .5,'review_atc'][:100]))

조립 관련 리뷰 갯수 : 20339
['배송 매우 빨랐구요 여자혼자 조립하기에도 다른상품보다 훨씬 수월하네오 화이트가 깨끗해보이고 세련되 보여요 같이 주문한 다른상품들도 만족입니다', '일단 생각보다 큽니다ㅋㅋ조립은 남자혼자 하기에는 무리없습니다. 여자는 조금 힘들수도 있을거 같아요. 사진과 똑같구요 찌그러진부분 없이 잘왔습니다~', '찌그러진게 많다더니  한가운데 저렇게 딱찌그러져서  펴지짇 않구 짜증나요 조립도 어렵진않은데  하나끼유면 하나빠지구 왠만하면 둘이하세요  여자혼자는 승질버림', '조립이 넘 어렵고 찌그러져있어요ㅠ', '조립이 좀 마지막에 어려웠는데가격 대비 괜찮은 것 같아요~~~~!!!!~~~~!!!!~', '저 160도안되는 여자사람인데 잘 조립했습니다 혼자! 여성분들 다 하실수있어요! 다들 걱정말구 시키세요! 물론 1시간정도 걸렸습니다...왕무거워요', '마지막 윗판 얹는 것 땜에 약간 애먹었지만 조립은 크게 어렵지 않았어요.문제는 문을 열 때마다 고정쇠가 빠져서 문을 확 열 수 없다는 점?이 좀 불편하긴 하지만 깔끔하고 예쁩니다.', '생각보다 커서 놀랐어요 근데 어렵기보단 좀 무거워서 여자 혼자 조립하긴 좀 낑낑거리기도 하고 시간도 좀 걸릴것 같긴 해요 그리고 상단판 가운데부분 찌그러져 와서 난감했어요', '조잡하고 조립설명서와 연결부품누락으로 빠른 발송요청했으나 나흘만에오고 설명서 또 누락되어 왔습니다.잘 좀 하시지ㅜ', '상판결합할때 설명서그림잘못보고 헷갈렸지만 이쁩니다.별도 도구없이 조립가능하구 무게 좀 있어서 안정적입니다. 가벼워서 흔들거리거나하면 어카지 했거든요', '조립하기 힘들어가지고서리 ㅋ 설치해놓으니 이쁘네요 전 tv스탠ㄷ로 사용안하고 공간이 있어 거기다 ?는데 좋네요', '생각보다 크지만 가성비 좋고 이쁩니다. 여자 혼자는 조립하기 좀 까다로웠을수도 있을거 같네용', '디자인은 화면과 같이 예뻐요~무광이라 더 맘에 들고요 조립도 무게감빼고는 쉬워요 단 철제용이고 무광이라서 긁힘이 심하니 조립시 주의해야함니다 그리고 상판

In [51]:
print("서비스(배송포함) 관련 리뷰 갯수 : {}".format(merge_df.loc[merge_df.tag_service_score > .5].shape[0]))
print(list(merge_df.loc[merge_df.tag_service_score > .5,'review_atc'][:100]))

서비스(배송포함) 관련 리뷰 갯수 : 34193
['다른 상품평에서 본것과 같이 찌그러짐이 장난아닙니다ㅠ펴서 조립을할지 교환을할지 반품을할지 고민중ㅠㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ다시받아도 안찌그러질거란 보장이 없어서그냥 조립했네요~특별히 많이 찌그러진게 상판 뒷쪽이라 사진처럽 웁니다ㅠ 티비 놓으니 표는 안나는데아래 셋탑박스 놓은부분이 울퉁불퉁합니다ㅡ멀리서 본 전체적인 모습은 이쁘구요ㅡ아래 수납도 많이 될듯하네요', '전화도안되고 상태도 안좋아요 완전 비추입니다', '아직 조립은 안했습니다~!!배송도 늦지않게 박스에 안전하게 포장되어 잘 도착했습니다!!!감사합니다✌️', '조잡하고 조립설명서와 연결부품누락으로 빠른 발송요청했으나 나흘만에오고 설명서 또 누락되어 왔습니다.잘 좀 하시지ㅜ', '진짜 반품신청 한지가 언젠데 물건도 조립도 안되는거 보내주고 반품도 2달이 되도 이제서야 연락오고  무슨 이런 업체가 다 있는건지 물건도 허접하고 처리도 허접하고 진짜 통화 되면 욕하고 싶을정도로 화가난다 부피도 큰데다가 카드 환불 처리도 해야 하는데 11번가도 화나고 이업체도 화나고 몇번을 연락해서  회수 해간다고  문자온게 또 일주일이  걸린단다 정말 열받고 짜증나고 절대 마켓비 물건 시킬일도 남들도 못사게 하고 싶다 사람들 후시사진 도 다 조작이 아닌가  하는 의구심도 생긴다 돈주고 다들 시켜서 후기 올린것 같다 이번에 빨리 처리 안해주면 진짜 가만히 안있는다', '디자인은 화면과 같이 예뻐요~무광이라 더 맘에 들고요 조립도 무게감빼고는 쉬워요 단 철제용이고 무광이라서 긁힘이 심하니 조립시 주의해야함니다 그리고 상판이 양쪽 찌그러져서 사진찍어 보내고 교환받아 다시 조립하는 번거로움이 있었네요그리고  배송기사님  배송예정일이라 하루종일 집에서 기다렸더니 저녁 9시 다되서 경비실에 맡겼다고 문자 하나오네요 가벼운 제품같으면 바쁘신가보다 하고 생각할텐데 여자혼자 못들어 담날저녁이나 되서야  신랑퇴근후 받아왔네요 홈쇼핑에서 주문하는건 집앞까지 배송이 기본아닌가요?

In [45]:
print("갯수 : {}".format(merge_df.loc[merge_df.tag_price_score > .5].shape[0]))
print(list(merge_df.loc[merge_df.tag_price_score > .5,'review_atc'][:100]))

갯수 : 15504
['물건 진짜 이 따위로 팔지 마세요 찌그러진 곳만 3곳인데 경미한 정도가 아닌데 배송중의 문제든 물건 출고시의 문제든 신경쓰세요 이 딴걸 물건이라고 파는게 이해가 안가네ㅡㅡ', '생각보다 크지만 가성비 좋고 이쁩니다. 여자 혼자는 조립하기 좀 까다로웠을수도 있을거 같네용', '배송은 빨라서 좋은데철제가 찌그러져서 왔어요.. 다른곳보다 저렴한게 이런이유때문에 그런건지 아무튼 우여곡절끝에 조립해서 사용합니다 가성비 갠찮어요', '위에 판 앞쪽이 좀 찌그러져서 왔어요교환하기 귀찮아서 쓰지만 짜증나네요그리고 앞에 열쇠를 항상 꽂아놓고 써야되네요 ㅋㅋㅋ당황싼 맛에 잠깐 쓴다고 생각해야 스트레스가없어요', '이케아꺼 구매할려다가 그나마 조금 저렴한 여기꺼 구매한거고.. 배송비만 2만원짜린데 진짜 장난치는것도 아니고;; 상판이 다 찌글짜글해서 조립하는데 애먹었습니다. 한쪽 문짝은 너덜거리네요.. 그냥 싼마이에 샀다고하고 그냥 쓰겠습니다만 이딴식으로 장사하지마세요.. 리뷰를 안보고 구매해서 후회중이네요;; 정상적으로 제품온 사람이 단 1명도 없는거 같네요ㅋㅋ 배송만 뒤지게빠르고 중국산대비 비싸고 제품품질은 최악 수준', '마켓비 제품은 완제품 아니면 절대 안삽니다. 구매하시려는 분들 이 글보고 참고하세요. 리뷰안남기려고 했는데 오늘 교환제품받아보고 어이없어서 글 쓰네요. 꼭 참고하세요. 일단 마켓비 제품 세개를 구매 했습니다. 11번가에서 한개 마켓비 공식쇼핑몰에서 한개 ㅈㅁㅋ에서 한개 이렇게 세개를 주문했는데요. 찌그러져서 오는건 기본입니다. 전 공구 다를줄알고 가구도 취미로 만들고 하는 남성이라서 그나마 찌그러진부분 펴가며 안맞는부분 맞춰가며 조립했는데요. 여자분들은 아마 찌그러진 상태와 부분에 따라 조립이 불가능하거나 힘드실겁니다. 이게 조립전에 찌그러진 상태를 확인해야 전체교환이 가능하다 명시되어 있는데요. 조립해야되는부분 미세한 찌그러짐은 발견하기가 쉽지 않습니다.  조립하게 되면 안맞는다는걸 인지하고 그때서야 알게되죠. 저는 상판한개가 보이는

### 감정 분석 - Doc2Vec
http://www.lifebloom.biz/2017/08/21/python%EC%9D%84-%ED%99%9C%EC%9A%A9%ED%95%9C-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EB%A7%88%EC%9D%B4%EB%8B%9D-9-%ED%85%8D%EC%8A%A4%ED%8A%B8-%EB%B6%84%EC%84%9D-%EA%B0%90%EC%84%B1-%EB%B6%84%EC%84%9Dsentiment-an-2/

https://www.kaggle.com/tj2552/sentiment-classification-in-5-classes-doc2vec

In [131]:
import gensim
from gensim.models import Doc2Vec
import numpy as np
from random import shuffle
from sklearn.linear_model import LogisticRegression

In [137]:
from gensim.models.doc2vec import TaggedDocument

In [278]:
def sampling_dataset(df):
    count = 5000
    class_df_sampled = pd.DataFrame(columns = ['score','tokens'])
    temp = []
    for c in df.score.unique():
        class_indexes = df[df.score == c].index
        random_indexes = np.random.choice(class_indexes, count, replace=False)
        temp.append(df.loc[random_indexes])
        
    for each_df in temp:
        class_df_sampled = pd.concat([class_df_sampled, each_df],axis=0)
    return class_df_sampled[['score','tokens']]

In [279]:
merge_df['score'] = merge_df.review_grade.apply(lambda x : x-1 if x>1 else 1)
df = sampling_dataset(merge_df)
df.reset_index(drop=True,inplace=True)

In [282]:
def label_sentences(df):
    labeled_sentences = []
    for index, row in df.iterrows():
        labeled_sentences.append(TaggedDocument(words=row.tokens,tags=['SENT_{}'.format(index)]))
    return labeled_sentences

In [283]:
sen = label_sentences(df)

In [ ]:
embedding_model = Word2Vec(sentences, 
                           size=100, # 100차원에서 연산하라
                           window=4, # 앞뒤로 4개까지 보라
                           min_count=50, # 코퍼스 내 출현 빈도가 50번 미만은 제외하라
                           workers=4, # cpu는 4개
                           iter=100, # 반복은 100번
                           sg=1) # 학습은 CBOW와 Skip-Gram 중 후자

In [306]:
def train_doc2vec_model(labeled_sentences):
    model = Doc2Vec(
        size=300,window=15,
        min_count=1,workers=4,
        iter=10,dm=1,
        alpha=0.025,min_alpha=0.025)
    model.build_vocab(labeled_sentences)
    for epoch in range(10):
        print(epoch)
        model.train(labeled_sentences,total_examples=len(labeled_sentences),epochs=10)
        model.alpha -= 0.002 
        model.min_alpha = model.alpha
    
    return model

In [307]:
model = train_doc2vec_model(sen)

0
1
2
3
4
5
6
7
8
9


In [310]:
def vectorize_comments(df,d2v_model):
    y = []
    comments = []
    for i in range(0,df.shape[0]):
        label = 'SENT_%s' %i
        comments.append(d2v_model.docvecs[label])
    df['vectorized_comments'] = comments
    
    return df

df = vectorize_comments(df,model)
print (df.head(2))

  score                                             tokens  \
0     4  [책, 놓는, 부분, 이, 두툼하, 니, .., 튼튼해, 보입니, 다, ., 색상,...   
1     4                                [이쁘, 고, 만족합, 니다, ㅋ]   

                                 vectorized_comments  
0  [-0.119919, 0.406116, 0.266393, 0.234491, -0.4...  
1  [0.198796, 0.280881, 0.198538, -0.384807, -0.3...  


In [317]:
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier as RFC
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pickle

def train_classifier(X,y):
    n_estimators = [200,400]
    min_samples_split = [2]
    min_samples_leaf = [1]
    bootstrap = [True]

    parameters = {'n_estimators': n_estimators, 'min_samples_leaf': min_samples_leaf,
                  'min_samples_split': min_samples_split,"bootstrap": bootstrap}

    clf = GridSearchCV(RFC(verbose=1,n_jobs=4), cv=4, param_grid=parameters)
    clf.fit(X, y)
    return clf

In [325]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df["vectorized_comments"].T.tolist(), df["score"].astype('category'), test_size=0.02, random_state=17)
classifier = train_classifier(X_train,y_train)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   15.6s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   16.3s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.2s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   15.4s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   16.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   15.5s
[Parallel(n_jobs=4)]: Done 200 out of 200 | elapsed:   16.0s finished
[Parallel(n_jobs=4)]: Done  42 tasks    

In [326]:
print(classifier.best_score_, "----------------Best Accuracy score on Cross Validation Sets")
print(classifier.score(X_test,y_test))

0.3976530612244898 ----------------Best Accuracy score on Cross Validation Sets
0.4325


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 400 out of 400 | elapsed:    0.1s finished


In [361]:
model.most_similar(positive='좋')

[('괜찮', 0.746297299861908),
 ('이', 0.45649123191833496),
 ('가', 0.4210606813430786),
 ('.', 0.41418004035949707),
 ('알', 0.4024951159954071),
 ('좀', 0.39754629135131836),
 ('..', 0.3963393568992615),
 ('조', 0.39242592453956604),
 ('놓', 0.38586440682411194),
 ('잘', 0.382917582988739)]

In [335]:
classifier.fit()

ValueError: Expected array-like (array or non-string sequence), got None